In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2018' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2018.csv
Avengers: Infinity War 2018-04-27
Black Panther 2018-02-16
Jurassic World: Fallen Kingdom 2018-06-22
Incredibles 2 2018-06-15
Aquaman 2018-12-21
Bohemian Rhapsody 2018-11-02
Venom 2018-10-05
Mission: Impossible - Fallout 2018-07-27
Deadpool 2 2018-05-18
Fantastic Beasts: The Crimes of Grindelwald 2018-11-16
Ant-Man and the Wasp 2018-07-06
Ready Player One 2018-03-29
The Meg 2018-08-10
Hotel Transylvania 3: Summer Vacation 2018-07-13
The Grinch 2018-11-09
Bumblebee 2018-12-21
A Star Is Born 2018-10-05
Rampage 2018-04-13
Mamma Mia! Here We Go Again 2018-07-20
Solo: A Star Wars Story 2018-05-25
Spider-Man: Into the Spider-Verse 2018-12-14
Fifty Shades Freed 2018-02-09
The Nun 2018-09-07
Peter Rabbit 2018-02-09
Mary Poppins Returns 2018-12-19
A Quiet Place 2018-04-06
Green Book 2018-11-16
Skyscraper 2018-07-13
Ocean's Eight 2018-06-08
Pacific Rim: Uprising 2018-03-23
Maze Runner: The Death Cure 2018-01-26
Tomb Raider 2018-03-16
Halloween 2018-10-19
Crazy Rich Asian